## Storing and Pulling Data from Databases with SQL

### Set Up
1.	Go to github 
2.	Download the 'Currency_Continent.csv', 'Currency_Map.csv' and 'ETFs.csv' files
3.	Move these files to a dedicated folder on your desktop or other location


### Needed Packages
1.	pandas
2.  sqlite3
3.  datetime

# Table of Contents

### SQL Overview
#### <a href='#1'>Introduction</a>

### Basics
#### <a href='#2'>Table Creation</a>
#### <a href='#3'>Table Modification</a>
* INSERT INTO
* DELETE FROM

#### <a href='#4'>The Basic SQL Statement Structure</a>

### Advanced
#### <a href='#5'>Joining Tables</a>
#### <a href='#6'>Table Aggregation by Groups</a>
#### <a href='#7'>IN and BETWEEN Keywords</a>

### Using Pandas
#### <a href='#9'>Loading CSV/Excel Data into SQL Database</a>

#### <a href='#15'>Weekly Readings/Videos</a>
#### <a href='#16'>Extra Practice</a>

<a id='1'></a>
## Introduction

![Meme](https://benjdd.com/assets/memes/sql-meme-1.jpg) 

Unlike most other programming languages, SQL is not designed for computations, instead SQL is used to interact and pull data out of data sets which can then be used by other languages to do analysis.  Additionally, SQL allows its users to grab data from any number of data sets simultaneously, so long as a relationship exists between them.  With SQL (pronounced like "sequel"), tables are the data organization structure. A table has a fixed number of named columns and a table row represents a single data record which contains one unique value for each column.  An example can be seen below in which a table represents a student's name, age and major with each row representing a different student.

|Name   | Age  | Major      |
|-------|------|------------|
|James  | 19   | Mathematics|
|Sarah  | 20   | Finance    |
|Isabel | 18   | Economics  |
|Alex   | 21   | Accounting |

First, we will start with the basics by importing 2 widely used packages into python, "sqlite3" which allows us to interact with SQL databases and "pandas" which is sort of like a souped-up version of excel in programming languages that works great with manipulating and presenting table data.

In [1]:
# sqlite3, pandas and datetime packages come with the Python/Anaconda installation
import sqlite3
import pandas as pd
import datetime as dt

# if this errors out, you can uncomment out these commands in a different cell:
#!conda install sqlite3
#!conda install pandas
#!conda install datetime

In [2]:
# open connnection to a db file stored locally on disk - if file doesn't exist it is created
connection = sqlite3.connect('lecture.db')

#Check if "lecture.db" exists using linux commands
!ls

Currency_Continent.csv ETFs.csv               lecture.db
Currency_Map.csv       SQL_Lecture.ipynb


In [3]:
# In order to run SQL commands with sqlite 3, we must create a cursor object
# that traverses the database to run sql commands execute them
cursor = connection.cursor()

<a id='2'></a>
## Table Creation

SQL can be used to create tables without existing data or to pull directly from existing databases.  Populating small tables of data by hand is inefficient for larger databases but works well with demonstrations.  You can create a table in SQL using statements in this format:

<code>
  CREATE TABLE [table_name] AS
  SELECT [val1] AS [column1], [val2] AS [column2], ... UNION
  SELECT [val3]             , [val4]             , ... UNION
  SELECT [val5]             , [val6]             , ...;
</code>

For example, if we wanted to create a table called resume_experience which contains columns for Years, City, Occupation, and Cost of Living you would write something like this:

<code>
  CREATE TABLE resume_experience AS
  SELECT 3 AS Years, Lexington AS City, Undergraduate AS Occupation, Low AS Cost_of_Living UNION
  SELECT 2,          Washington D.C.,   Research Assistant,          High                  UNION
  SELECT 1,          Berkeley,          Masters,                     High                  UNION
  SELECT 1,          St. Louis,         Financial Analyst,           Low;
</code>

In [4]:
# Check that we are working with an empty db
cursor.execute("DROP TABLE IF EXISTS resume_experience;")

# We can define long SQL commands within three quotes

sql_command = """
CREATE TABLE resume_experience AS
  SELECT 3 AS Years, "Lexington" AS City, "Undergraduate" AS Occupation, "Low" AS Cost_of_Living UNION
  SELECT 2,          "Washington D.C.",   "Research Assistant",          "High"                  UNION
  SELECT 1,          "Berkeley",          "Masters",                     "High"                  UNION
  SELECT 1,          "St. Louis",         "Financial Analyst",           "Low";"""

# In order to run SQL command on the databse file we have to execute them with the cursor
cursor.execute(sql_command)

# Load the sql table into a pandas dataframe for aesthetics
# The function's arguments are (sql_query_to_run, connection_to_database)
pd.read_sql_query('SELECT * FROM resume_experience',con = connection)

,Years,City,Occupation,Cost_of_Living
0,1,Berkeley,Masters,High
1,1,St. Louis,Financial Analyst,Low
2,2,Washington D.C.,Research Assistant,High
3,3,Lexington,Undergraduate,Low


In [5]:
# Check that we are working with an empty db
cursor.execute("DROP TABLE IF EXISTS teachers;")

connection = sqlite3.connect('lecture.db')
cursor = connection.cursor()

# We can also create empty databases to populate later
# VARCHAR string that can be up to X characters
# CHAR has to be X characters
# INTEGER contains numerical data without decimal points

#The table and column names must start with a letter and can be followed by letters, numbers, or underscores 
#- not to exceed a total of 30 characters in length. 
#Do not use any SQL reserved keywords as names for tables or column names 
#(such as "select", "create", "insert", etc).

sql_command = """
CREATE TABLE teachers (
Staff_number INTEGER PRIMARY KEY,
Fname VARCHAR(20),
Lname VARCHAR(30),
Subject CHAR(20),
City CHAR(20));"""

cursor.execute(sql_command)

<a id='3'></a>
## Table Modification

### INSERT INTO

![InsertMeme](https://memegenerator.net/img/instances/66765869/writing-insert-based-on-join-sql-queries.jpg) 

After a table has been created, a user can add more rows manually using the INSERT INTO statement.

<code>
INSERT INTO [table] ([column1], [column2], ...)
VALUES ([value1], [value2], ...), ([value1], [value2], ...);
</code>

- If there exists a value for each row, column pairing then you do not have to specify column names.
- For columns where a value is not specified, the default value will be used if available (specified by the creator) otherwise it will appear empty.

In [6]:
sql_command = '''
INSERT INTO teachers
(Staff_number, Fname, Lname, Subject, City)
VALUES
(300, "Wendy", "Liu", "Finance", "Lexington");'''

cursor.execute(sql_command)

In [7]:
# Assume we already have a lecture.db sqlite database file with a Table called teachers (which we do)
# Add several values with sqlite3 and string formatting

# list of tuples we want to add to the database
teacher_data = [ (479, "Chris", "Bollinger", "Economics", "Lexington"), \
              (327, "Adib", "Bagh", "Mathematics", "Lexington"),  \
              (202, "Shelly", "Antonio", "Economics", "Washington D.C."), \
             (275, "Wayne", "Delker", "Industrial Engineering", "Berkeley"), \
               (115, "Teresa", "Glenn", "Mathematics", "St. Louis")]

for s in teacher_data:
    
    # define string to be formatted, name format placeholders within curly brackets
    format_str = '''
    INSERT INTO teachers (Staff_number, fname, lname, Subject, City)    
    VALUES ("{number}", "{first}", "{last}", "{subject}", "{city}");
    '''

    # define SQL command
    sql_command = format_str.format(number=s[0],first=s[1], last=s[2], 
                                subject=s[3], city = s[4]) 
    
    cursor.execute(sql_command)
    
connection.commit()
pd.read_sql_query('SELECT * FROM teachers',con = connection)

,Staff_number,Fname,Lname,Subject,City
0,115,Teresa,Glenn,Mathematics,St. Louis
1,202,Shelly,Antonio,Economics,Washington D.C.
2,275,Wayne,Delker,Industrial Engineering,Berkeley
3,300,Wendy,Liu,Finance,Lexington
4,327,Adib,Bagh,Mathematics,Lexington
5,479,Chris,Bollinger,Economics,Lexington


### DELETE FROM

The delete statement is used to delete records or rows from the table.
<code>
delete from "tablename"

where "columnname" 
  OPERATOR "value" 
[and|or "column" 
  OPERATOR "value"];
</code>
Anything above in brackets is an optional statement.


The example below deletes all rows from the table named "system" that has rows in which the column "id" is equal to "642a".
<code>    
delete from system
  where id = '642a';
</code>

To delete an entire record/row from a table, enter "delete from" followed by the table name, followed by the where clause which contains the conditions to delete. Rembemer... if you leave off the where clause, ALL RECORDS WILL BE DELETED.

In [8]:
# never forget to commit your queries if you have updated the database
# and close the connection when you're done

connection.commit() 

connection.close()

<a id='4'></a>
## The Basic SQL Statement Structure

![Santa](https://static.displate.com/392x280/displate/2021-01-23/2d0c229cf73a99888c63d5f84833bbdb_30f028b3a61b5650c8e57f1084529e6c.jpg) 

<code>
SELECT [columns] 
FROM [tables] 
WHERE [condition] 
GROUP BY [columns]
HAVING [condition]
ORDER BY [columns] 
LIMIT [limit];
</code>


Let's break down this statement:

<b>SELECT</b> [columns] tells SQL that we want to include the given columns in our output table; [columns] is a comma-separated list of column names, and * can be used to select all columns

<b>FROM</b> [table] tells SQL that the columns we want to select are from the given table; see the joins section to see how to select from multiple tables

<b>WHERE</b> [condition] filters the output table by only including rows whose values satisfy the given [condition], a boolean expression

<b>GROUP BY</b> combines those rows into groups based on these [columns]

<b>HAVING</b> Filters groups, is used when the WHERE keyword cannot be used with aggregate functions.  ie: HAVING count(variable) > 15

<b>ORDER BY</b> [columns] orders the rows in the output table by the given comma-separated list of columns

<b>LIMIT</b> [limit] limits the number of rows in the output table by the integer [limit]

Expressions in the SELECT, WHERE, and ORDER BY clauses can contain one or more of the following operators:

- comparison operators: =, >, <, <=, >=, <> or != ("not equal")
- boolean operators: AND, OR
- arithmetic operators: +, -, *, /
- concatenation operator: ||

Note: We capitalize SQL keywords purely because of style convention. It makes queries much easier to read, though they will still work if you don't capitalize keywords.

<i>source: https://inst.eecs.berkeley.edu/~cs61a/sp19/lab/lab12/<i>

In [9]:
connection = sqlite3.connect('lecture.db')
cursor = connection.cursor()

#The City and Order by variables are not included in the output but still manipulate the dataset

sql_statement = """
SELECT Staff_number, Lname, Subject
FROM teachers
WHERE City='Lexington'
ORDER BY Fname;
"""

pd.read_sql_query(sql_statement,con = connection)

,Staff_number,Lname,Subject
0,327,Bagh,Mathematics
1,479,Bollinger,Economics
2,300,Liu,Finance


## -------------PRACTICE-------------

1. Create a table of your top 5 movies or tv show with when the movie was probably released, your rating out of 10 for it and the genre.  Name this table "Movies".

2. Create another table related to question 1 but has the movie/tv show name as one column name and another column for character name using python lists and the insert into command.  Please have at least 2 characters for every movie/tv show that us.  Name this table "Characters".

3. Select your top 2 rated movies using an order by and limit command from the "Movies" table.

<a id='5'></a>
## Joining Tables

![SeriousJoin](https://learnsql.com/blog/learn-and-practice-sql-joins/2.png)

<code>
SELECT [columns]
FROM [table1] {as [new_table_name]}, [table2] {as [new_table_name2]}, ... 
WHERE [condition], [condition2], ...
ORDER BY [columns]
LIMIT</code>

<i>Note, you do not have to rename each table to a new table name.</i>

The example below the tables are renamed a and b because the tables include variables that exist in both databases.  The names a and b were chosen because they are shorter than their actual names to save time.  When referencing the column names, use the dot notation [table_name].[column name] to distinguish which columns are from which table.


![FunnyJoin](https://i.redd.it/9tsgqvarse821.jpg) 

In [10]:
join_statement = """
SELECT * FROM resume_experience as a LEFT JOIN teachers as b 
on a.City = b.City;
"""

pd.read_sql_query(join_statement,con = connection)

,Years,City,Occupation,Cost_of_Living,Staff_number,Fname,Lname,Subject,City
0,1,Berkeley,Masters,High,275,Wayne,Delker,Industrial Engineering,Berkeley
1,1,St. Louis,Financial Analyst,Low,115,Teresa,Glenn,Mathematics,St. Louis
2,2,Washington D.C.,Research Assistant,High,202,Shelly,Antonio,Economics,Washington D.C.
3,3,Lexington,Undergraduate,Low,327,Adib,Bagh,Mathematics,Lexington
4,3,Lexington,Undergraduate,Low,479,Chris,Bollinger,Economics,Lexington
5,3,Lexington,Undergraduate,Low,300,Wendy,Liu,Finance,Lexington


In [11]:
# Default join if not specified is an "inner join"
join_statement = """
SELECT * FROM resume_experience as a, teachers as b 
WHERE a.City = b.City AND Staff_number < 320
ORDER BY Cost_of_Living, Years DESC;
"""

pd.read_sql_query(join_statement,con = connection)

,Years,City,Occupation,Cost_of_Living,Staff_number,Fname,Lname,Subject,City
0,2,Washington D.C.,Research Assistant,High,202,Shelly,Antonio,Economics,Washington D.C.
1,1,Berkeley,Masters,High,275,Wayne,Delker,Industrial Engineering,Berkeley
2,3,Lexington,Undergraduate,Low,300,Wendy,Liu,Finance,Lexington
3,1,St. Louis,Financial Analyst,Low,115,Teresa,Glenn,Mathematics,St. Louis


<a id='6'></a>
## Table Aggregation by Groups

![FunnyGroup](https://cdn.buttercms.com/51DnKV3OSsix08qjx1NA) 

Table agregation allows us to accumulate values via aggregator statements across rows in our SQL database by grouping the data together by one or more qualitative variables.  The below functions are some examples of possible aggregate functions:

- the sum of the values in the column for that grouping (SUM)
- the maximum value (MAX),
- the minimum value (MIN),
- the number of rows in the group (COUNT),
- the average over all of the values (AVG),

Whenever an aggregation is done, a GROUP BY clause must be used with variable(s) in the SQL statement along with the aggregator function being used on variable(s) in the SELECT clause.

In [12]:
group_statement = """
SELECT City, COUNT(City) as Number_of_Teachers
FROM teachers
GROUP BY City
;
"""

pd.read_sql_query(group_statement,con = connection)

,City,Number_of_Teachers
0,Berkeley,1
1,Lexington,3
2,St. Louis,1
3,Washington D.C.,1


<a id='7'></a>
## IN and BETWEEN Keywords

The IN conditional operator is really a set membership test operator. That is, it is used to test whether or not a value (stated before the keyword IN) is "in" the list of values provided after the keyword IN.

<code>SELECT col1, SUM(col2)
FROM "list-of-tables"
WHERE col3 IN (list-of-values);</code>

OR
    
<code>SELECT col1, SUM(col2)
FROM "list-of-tables"
WHERE col3 BETWEEN value1 
AND value2; </code>

An example can be seen below:

<code>SELECT employeeid, lastname, salary
FROM employee_info
WHERE lastname IN ('Hernandez', 'Jones', 'Roberts', 'Ruiz');</code>
 

This statement will select the employeeid, lastname, salary from the employee_info table where the lastname is equal to either: Hernandez, Jones, Roberts, or Ruiz. It will return the rows if it is ANY of these values.

The IN conditional operator can be rewritten by using compound conditions using the equals operator and combining it with OR - with exact same output results:

<code>SELECT employeeid, lastname, salary
FROM employee_info
WHERE lastname = 'Hernandez' OR lastname = 'Jones' OR lastname = 'Roberts'
OR lastname = 'Ruiz';</code>
 

As you can see, the IN operator is much shorter and easier to read when you are testing for more than two or three values.

You can also use NOT IN to exclude the rows in your list.

The BETWEEN conditional operator is used to test to see whether or not a value (stated before the keyword BETWEEN) is "between" the two values stated after the keyword BETWEEN.

For example:

<code>SELECT employeeid, age, lastname, salary
FROM employee_info
WHERE age BETWEEN 30 AND 40;</code>

source: https://www.sqlcourse2.com/setoper.html

In [13]:
in_statement = """
SELECT City, Cost_of_Living
FROM resume_experience
WHERE Cost_of_Living in ('High')
GROUP BY City
;
"""

pd.read_sql_query(in_statement,con = connection)

,City,Cost_of_Living
0,Berkeley,High
1,Washington D.C.,High


In [14]:
between_statement = """
SELECT City, Years
FROM resume_experience
WHERE Years between 2 and 5
GROUP BY City
;
"""

pd.read_sql_query(between_statement,con = connection)

,City,Years
0,Lexington,3
1,Washington D.C.,2


## -------------PRACTICE-------------

1. Join the "Movies" and "Characters" tables together by the movie id you used.

2. Using the table you created above, display the count of the movies you had by genre.

3. Using that same table, display the total sum of the ratings and the count of characters for the movies in every genre.

<a id='8'></a>
## Loading CSV/Excel Data into SQL Database
![FunnyFunds](https://i.pinimg.com/736x/14/5c/32/145c32bf3771c7a5e36cef28b2f673c8.jpg) 

In [15]:
#https://www.kaggle.com/stefanoleone992/mutual-funds-and-etfs

#Used for timer
start = dt.datetime.now()
#Number of rows to read in at a time
chunksize = 500

#Used to increment
j = 0
index_start = 1

for df in pd.read_csv('ETFs.csv', chunksize=chunksize, iterator=True, encoding='utf-8'):
    
    # Remove spaces from columns
    df = df.rename(columns={c: c.replace(' ', '') for c in df.columns})

    #Index the data
    df.index += index_start    
    j+=1

    df.to_sql('ETF_data', connection, if_exists='replace') # name of SQL table, connection, append
    print('{} seconds: completed {} rows'.format((dt.datetime.now() - start).seconds, j*chunksize))
    index_start = df.index[-1] + 1 # update index start
print('done')

0 seconds: completed 500 rows
0 seconds: completed 1000 rows
0 seconds: completed 1500 rows
0 seconds: completed 2000 rows
0 seconds: completed 2500 rows
done


In [16]:
sql_statement = """
SELECT *
FROM ETF_data
;
"""

pd.read_sql_query(sql_statement,con = connection)

,index,fund_name,fund_extended_name,category,fund_family,net_assets,ytd_return,fund_yield,legal_type,investment,...,fund_sharpe_ratio_5years,category_sharpe_ratio_5years,fund_sharpe_ratio_10years,category_sharpe_ratio_10years,fund_treynor_ratio_3years,category_treynor_ratio_3years,fund_treynor_ratio_5years,category_treynor_ratio_5years,fund_treynor_ratio_10years,category_treynor_ratio_10years
0,7001,SVXY,ProShares Short VIX Short-Term Futures ETF,Volatility,ProShares,441210000.0,28.78,0.00,Exchange Traded Fund,None,...,0.36,NaN,0.0,NaN,20.9,NaN,17.55,None,0.00,NaN
1,7002,SYE,SPDR MFS Systematic Core Equity ETF,Large Blend,SPDR State Street Global Advisors,40100000.0,16.52,1.54,Exchange Traded Fund,Blend,...,0.99,1.34,0.0,0.48,12.68,10.38,11.34,15.73,0.00,6.54
2,7003,SYG,SPDR MFS Systematic Growth Equity ETF,Large Growth,SPDR State Street Global Advisors,32460000.0,17.78,1.01,Exchange Traded Fund,Growth,...,0.92,1.34,0.0,0.58,9.82,10.86,10.71,16.11,0.00,8.25
3,7004,SYLD,Cambria Shareholder Yield ETF,Mid-Cap Value,CAMBRIA ETF TRUST,109950000.0,15.69,2.49,Exchange Traded Fund,Value,...,0.54,1.37,0.0,0.44,7.45,11.88,5.98,17.57,0.00,6.04
4,7005,SYV,SPDR MFS Systematic Value Equity ETF,Large Value,SPDR State Street Global Advisors,33020000.0,13.82,2.06,Exchange Traded Fund,Value,...,0.80,1.38,0.0,0.43,11.74,11.86,9.23,17.48,0.00,5.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,7348,ZBIO,ProShares UltraPro Short Nasdaq Biotechnology,Trading - Inverse Equity,ProShares,4630000.0,-33.01,0.14,Exchange Traded Fund,None,...,0.00,NaN,0.0,NaN,9.47,NaN,0.00,None,0.00,NaN
348,7349,ZIV,VelocityShares Daily Inverse VIX Medium-Term ETN,Volatility,Credit Suisse AG,99080000.0,25.94,0.00,Exchange Traded Fund,None,...,0.57,NaN,0.0,NaN,-47.82,NaN,-30.81,None,0.00,NaN
349,7350,ZMLP,Direxion Zacks MLP High Income Index Shares,Energy Limited Partnership,Direxion Funds,50920000.0,18.43,11.78,Exchange Traded Fund,Value,...,-0.45,0.19,0.0,NaN,3.18,-5.70,-8.02,1.89,0.00,NaN
350,7351,ZROZ,PIMCO 25+ Year Zero Coupon U.S. Treasury Index...,Long Government,PIMCO,199740000.0,3.21,2.74,Exchange Traded Fund,None,...,0.49,0.54,0.0,0.69,0.07,3.03,1.36,1.53,0.00,2.58


In [17]:
#Used for timer
start = dt.datetime.now()
#Number of rows to read in at a time
chunksize = 500

#Used to increment
j = 0
index_start = 1

for df in pd.read_csv('Currency_Map.csv', chunksize=chunksize, iterator=True, encoding='utf-8'):
    
    
    # Remove spaces from columns
    
    df = df.rename(columns={c: c.replace(' ', '') for c in df.columns})
    
    #Index the data
    df.index += index_start    
    j+=1
    df.to_sql('Currency_Map', connection, if_exists='replace') # name of SQL table, connection, append
    print('{} seconds: completed {} rows'.format((dt.datetime.now() - start).seconds, j*chunksize))
    index_start = df.index[-1] + 1 # update index start
print('done')

0 seconds: completed 500 rows
done


In [18]:
sql_statement = """
SELECT *
FROM Currency_Map
;
"""

pd.read_sql_query(sql_statement,con = connection)

,index,Currency,USDConversion
0,1,GBP,1.30
1,2,EUR,1.18
2,3,CAD,0.75


In [19]:
#Used for timer
start = dt.datetime.now()
#Number of rows to read in at a time
chunksize = 500

#Used to increment
j = 0
index_start = 1

for df in pd.read_csv('Currency_Continent.csv', chunksize=chunksize, iterator=True, encoding='utf-8'):
    
    
    # Remove spaces from columns
    
    df = df.rename(columns={c: c.replace(' ', '') for c in df.columns})
    
    #Index the data
    df.index += index_start    
    j+=1
    df.to_sql('Currency_Continent', connection, if_exists='replace') # name of SQL table, connection, append
    print('{} seconds: completed {} rows'.format((dt.datetime.now() - start).seconds, j*chunksize))
    index_start = df.index[-1] + 1 # update index start
print('done')

0 seconds: completed 500 rows
done


In [20]:
sql_statement = """
SELECT *
FROM Currency_Continent
;
"""

pd.read_sql_query(sql_statement,con = connection)

,index,Currency,Continent
0,1,GBP,Europe
1,2,EUR,Europe
2,3,CAD,North America


## -------------PRACTICE-------------
1. Query the fund_yield and fund_name for all PIMCO funds.

In [ ]:
sql_statement = """
SELECT *
FROM ETF_data...
;
"""

pd.read_sql_query(sql_statement,con = connection)

2. Using the Currency_Map table, find the conversion rate from between USD and GBP.  Please note that this table provides USD per unit of foreign currency.  Your final answer should be a number.

In [ ]:
sql_statement = """
SELECT * 
From Currency_Map...;
"""
conversion_rate = pd.read_sql_query(sql_statement,con = connection).iloc[0][0]
print(conversion_rate)



3. Using the previous solution, query fund_name and net_assets from ETF_Data in GBP.

In [ ]:
sql_statement = """
SELECT *
FROM ETF_data...
;
"""

pd.read_sql_query(sql_statement,con = connection)

4. Query all fund_name and fund_treynor_ratio_5years where fund_treynor_ratio_5years is greater than 2.  Sort the query in ascending order.

In [ ]:
sql_statement = """
SELECT *
FROM ETF_data...
;
"""

5. Create a query that returns the USD conversion and continent for all matching currencies between Currency_Map and Currency_Continent. 

In [ ]:
join_statement = """
SELECT * FROM Currency_Map as a, Currency_Continent as b 
WHERE ...;
"""

pd.read_sql_query(join_statement,con = connection)

6. Find the total of net_assets for all ProShares ETF's.  Your answer should be a number.

In [ ]:
sql_statement = """
SELECT * 
From ETF_data...;
"""
total_net_assets = pd.read_sql_query(sql_statement,con = connection).iloc[0][0]
print(total_net_assets)

In [ ]:
connection.commit() 

connection.close()

<a id='15'></a>
# Weekly Readings/Videos

https://www.forbes.com/sites/kashmirhill/2012/02/16/how-target-figured-out-a-teen-girl-was-pregnant-before-her-father-did/?sh=298bbc656668

https://www.forbes.com/sites/bernardmarr/2018/05/21/how-much-data-do-we-create-every-day-the-mind-blowing-stats-everyone-should-read/?sh=4ef6808160ba

https://www.youtube.com/watch?v=JAO_3EvD3DY

https://onlinedsa.merrimack.edu/nba-analytics-changing-basketball/

https://www.thespax.com/nba/analyzing-nba-spacing-with-player-tracking-data/

<a id='16'></a>
# Extra Practice

https://sqlzoo.net/wiki/SELECT_basics

https://www.w3schools.com/sql/sql_exercises.asp